In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


## Part barra M31 GADGET

In [19]:
import h5py
import rotation as rot
import tenform as ten
from numpy import linalg
import barstrength as strng

h=0.732
G = 4.299e-6
a0=1
H0 = h*100
omega_lambda=0.716
omega_matter=0.1277/(h**2.)
omega0 = omega_lambda + omega_matter

atime = np.loadtxt('/home/ornela/SimCLUES/redshift_outputs.txt')
aexp = atime[:,2]
aexp = aexp[::-1]

data = np.loadtxt('/home/ornela/PYTHON/Barras_GdGs/Barras_Gd/_data/M31_lbarvstime.dat')
lbar1 = data[:,1]
lbar2 = data[:,2]



snap = h5py.File('/home/ornela/SimCLUES/outputs_1/snap_496.h5py', 'r')
    
cm   = snap['subhalo_000/Center'].value
r200 = snap['subhalo_000/R200'].value

pstr = snap['subhalo_000/Str/Coordinates'].value
mstr = snap['subhalo_000/Str/Masses'].value
vel  = snap['subhalo_000/Str/Velocities'].value

z = a0/aexp[0] - 1.
Ht = H0*np.sqrt(omega_lambda+(1-omega0)*(1+z)**2+omega_matter*(1+z)**3)

#---aca paso las coordenadas respecto al centro de la galaxia------
xstr = (pstr[:,0]-cm[0])*aexp[0]/h
ystr = (pstr[:,1]-cm[1])*aexp[0]/h
zstr = (pstr[:,2]-cm[2])*aexp[0]/h
rstr = np.sqrt(xstr**2+ystr**2+zstr**2)

v_x = vel[:,0] *np.sqrt(aexp[0]) + Ht *xstr/1000.
v_y = vel[:,1] *np.sqrt(aexp[0]) + Ht *ystr/1000.
v_z = vel[:,2] *np.sqrt(aexp[0]) + Ht *zstr/1000.
#----------------------masas----------------------------
mstr = mstr/h

r200 = r200*aexp[0]/h

#         print 'r200 =', r200
rgal=0.15*r200

limit    = np.where(rstr<rgal)
r_sort   = np.sort(rstr[limit])
r_indice = np.argsort(rstr[limit])

Mgal_str = cumsum((mstr[limit])[r_indice])
M_gal = Mgal_str[-1]

razon      = Mgal_str/M_gal
noventa,   = np.where(razon < 0.9)
cincuenta, = np.where(razon < 0.5)

r90 = r_sort[noventa][-1]
r50 = r_sort[cincuenta][-1]
#--------------------------------------------------------------       

veloc, = np.where(rstr < r50)

vxcm = sum(mstr[veloc]*v_x[veloc])/sum(mstr[veloc])
vycm = sum(mstr[veloc]*v_y[veloc])/sum(mstr[veloc])
vzcm = sum(mstr[veloc]*v_z[veloc])/sum(mstr[veloc])

#velocidades de las estrellas respecto del centro de masa de la galaxia
vx = v_x - vxcm
vy = v_y - vycm
vz = v_z - vzcm

#-------------------------------------------------------------------------------------
#hago la rotación de las coordenadas respecto del momento angular    

e1x,e2x,e3x,e1y,e2y,e3y,e1z,e2z,e3z = rot.rot1(mstr,xstr,ystr,zstr,vx,vy,vz,3*aexp[isnap])

xn = e1x*xstr + e1y*ystr + e1z*zstr
yn = e2x*xstr + e2y*ystr + e2z*zstr
zn = e3x*xstr + e3y*ystr + e3z*zstr
vxn = e1x*vx + e1y*vy + e1z*vz
vyn = e2x*vx + e2y*vy + e2z*vz
vzn = e3x*vx + e3y*vy + e3z*vz

corte, = np.where(rstr < 3*aexp[0])

A2max, rmax, phimax = strng.a2max(mstr[corte], xn[corte], yn[corte], 20)

xx = xn*cos(phimax)+yn*sin(phimax)
yy = xn*-sin(phimax)+yn*cos(phimax)
zz = zn


#----para lbar1--------------------------------------------------------------
limit1,=np.where(rstr < lbar1[0])
tensor = ten.tenfr(xx[limit1], yy[limit1], zz[limit1], mstr[limit1])

#queremos sacar los autovalores
matriz = linalg.eig(tensor)
autov  = matriz[0]
asort  = sort(autov)
a1 = asort[2]
b1 = asort[1]
c1 = asort[0]

#----para lbar2--------------------------------------------------------------
limit2,=np.where(rstr < lbar2[0])
tensor = ten.tenfr(xx[limit2], yy[limit2], zz[limit2], mstr[limit2])

#queremos sacar los autovalores
matriz = linalg.eig(tensor)
autov  = matriz[0]
asort  = sort(autov)
a2 = asort[2]
b2 = asort[1]
c2 = asort[0]

print 'lbar1=', lbar1[0],'a1=',a1,'b1=', b1, 'c1=',c1
print 'lbar2=', lbar2[0],'a2=',a2,'b2=', b2, 'c2=',c2

lbar1= 2.116138 a1= 0.539376 b1= 0.27665 c1= 0.183974
lbar2= 1.542296 a2= 0.553309 b2= 0.264433 c2= 0.182258
